# LA Metro Bike System (Bike Sharing System)

Link the Google Drive with the Colab, to access data files.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Installing the libraries required.

In [1]:
import os
from os import listdir, makedirs, remove
import os.path as osp
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from numpy.random import Generator, PCG64

### Data Processing and Transformation 
This process consists of functions developed to faciliate creation of the desired dataset.

1. "time_bins" function creates a dataframe of start date time and end date time with interval size and returns it, to where the function is called.

In [2]:
def time_bins(start_date, end_date, interval_size):
  
    start = datetime.strptime(start_date, '%Y/%m/%d')
    end = datetime.strptime(end_date, '%Y/%m/%d')
    gap = timedelta(minutes=interval_size)

    interval_id = 0
    outer_limit = start
    vals = []
    while outer_limit < end:
        inner_limit = outer_limit
        outer_limit += gap
        i = [interval_id, inner_limit.strftime('%d/%m/%Y %H:%M'),
                      outer_limit.strftime('%d/%m/%Y %H:%M')]
        vals.append(pd.Series(i))
        interval_id += 1

    df_out = pd.DataFrame(vals)
    df_out = df_out.rename(columns={0: 'interval_id', 1: 'inner_limit', 2: 'outer_limit'})
    df_out['inner_limit'] = pd.to_datetime(df_out['inner_limit'], dayfirst=True)
    df_out['outer_limit'] = pd.to_datetime(df_out['outer_limit'], dayfirst=True)

    return df_out

2. "format_data" function loads the bike trip data, one file at a time and performs pre-preprocessing on it. And at end combines it with the time interval dataset obtained from "time_bins" function to develop transformed dataset and weights for graph.

In [3]:
def format_data(file_path, tinterval, interval_size, duration_upper, station_exclude, station_const_ids):
  
    def round_minutes(dt, direction, resolution):
        new_minute = (dt.minute // resolution + (1 if direction == 'up' else 0)) * resolution
        return dt + timedelta(minutes=new_minute - dt.minute)

    # Read bike trip data and load it into a pandas dataframe.
    df_raw = pd.read_csv(file_path, low_memory= False)
    df_raw['End Date'] = pd.to_datetime(df_raw['end_time'], dayfirst=True)
    df_raw['Start Date'] = pd.to_datetime(df_raw['start_time'], dayfirst=True)
    df_raw = df_raw.rename(columns={'end_station' : 'EndStation Id', 'start_station': 'StartStation Id', 'duration': 'Duration'})
    df_raw = df_raw[~df_raw['EndStation Id'].isna()]
    df_raw['EndStation Id'] = df_raw['EndStation Id'].astype(int)
    df_raw['StartStation Id'] = df_raw['StartStation Id'].astype(int)
    df_raw = df_raw.loc[df_raw['Duration'] < duration_upper]

    # Check to confirm no unknown station value is present in the data which is not recorded in bike station details dataset.
    s_diff = set(df_raw['StartStation Id'].to_list()) - station_const_ids
    e_diff = set(df_raw['EndStation Id'].to_list()) - station_const_ids
    if len(s_diff) > 0:
        raise RuntimeError('In file {} unknown start station IDs: {}'.format(file_path, s_diff))
    if len(e_diff) > 0:
        raise RuntimeError('In file {} unknown end station IDs: {}'.format(file_path, e_diff))

    # Remove unwanted stations like virtual stations
    df_raw = df_raw.loc[~df_raw['StartStation Id'].isin(station_exclude)]
    df_raw = df_raw.loc[~df_raw['EndStation Id'].isin(station_exclude)]

    # Count the number of bike trips made for station-to-station.
    df_graph_weight = df_raw.groupby(['StartStation Id', 'EndStation Id']).size()

    # Maps time of bike trip to a starting time of the interval, as defined by the dataset created by time_bins().
    df_raw['End Date Lower Bound'] = df_raw['End Date'].apply(round_minutes,
                                                              **{'direction': 'down', 'resolution': interval_size})
    df_raw['Start Date Lower Bound'] = df_raw['Start Date'].apply(round_minutes,
                                                                  **{'direction': 'down', 'resolution': interval_size})

    
    #Merge df_raw dataset with time interval dataset obtained from the time_bins().
    df1 = pd.merge(df_raw, tinterval, left_on='End Date Lower Bound', right_on='inner_limit').drop(
        columns=['inner_limit', 'outer_limit'])
    df1 = df1.rename({'interval_id': 'End Date ID'}, axis=1)
    df1 = pd.merge(df1, tinterval, left_on='Start Date Lower Bound', right_on='inner_limit').drop(
        columns=['inner_limit', 'outer_limit'])
    df1 = df1.rename({'interval_id': 'Start Date ID'}, axis=1)
    df1 = df1.drop(columns=['bike_id', 'Duration', 'passholder_type','plan_duration','start_lon','start_lat',
                          'End Date','Start Date', 'trip_route_category','bike_type','end_lat', 'end_lon',
                          'End Date Lower Bound', 'Start Date Lower Bound'])
    # Raises an error
    if len(df1) == 0:
        raise RuntimeError('Incorrect time intervals for file {}'.format(file_path))

    g_arrival = df1.groupby(by=['EndStation Id', 'End Date ID'])
    df_arrival = g_arrival.size()
    g_departure = df1.groupby(by=['StartStation Id', 'Start Date ID'])
    df_departure = g_departure.size()

    # Zero is substituted in place of records with missing value of time ids,  
    s1 = df_arrival.index.get_level_values('EndStation Id').unique()
    s2 = df_departure.index.get_level_values('StartStation Id').unique()
    s_index = s1.union(s2)
    t1 = df_arrival.index.get_level_values('End Date ID').min()
    t2 = df_departure.index.get_level_values('Start Date ID').max()
    t_index = pd.Int64Index(list(range(t1, t2 + 1)), dtype='int64')
    new_index = pd.MultiIndex.from_product([s_index, t_index],
                                           names=['station_id', 'time_id'])
    df_arrival = pd.DataFrame(df_arrival.reindex(new_index, fill_value=0), columns=['docks_demand'])
    df_departure = pd.DataFrame(df_departure.reindex(new_index, fill_value=0), columns=['bikes_demand'])

    df_all = df_arrival.join(df_departure, how='outer')

    return df_all, df_graph_weight


In [4]:
def merge_data(file_name_root, max_file_index, t_interval):

    drop_index = None
    df_all = []

    #iterating every files to remove duplicate element
    for ind in range(max_file_index):
        df = pd.read_csv('{}_{}.csv'.format(file_name_root, ind), index_col=(0, 1))
        for ind_ in range(ind + 1, max_file_index):
            print('Processing file pair {}-{} of {}'.format(ind, ind_, max_file_index))
            df1 = pd.read_csv('{}_{}.csv'.format(file_name_root, ind_), index_col=(0, 1))
            df2 = df.join(df1, how='inner', lsuffix='_0', rsuffix='_1')

            # If time-place coordinate overlap found put coordinates in new common file
            if len(df2) > 0:
                df2['docks_demand'] = df2['docks_demand_0'] + df2['docks_demand_1']
                df2['bikes_demand'] = df2['bikes_demand_0'] + df2['bikes_demand_1']
                df2 = df2.drop(['docks_demand_0', 'docks_demand_1', 'bikes_demand_0', 'bikes_demand_1'], axis=1)
                df_all.append(df2)

                if drop_index is None:
                    drop_index = df2.index
                else:
                    drop_index = drop_index.union(df2.index)

    # Drops duplicatte records and the collective data from all the files is saved.
    for ind in range(max_file_index):
        df = pd.read_csv('{}_{}.csv'.format(file_name_root, ind), index_col=(0, 1))

        if not drop_index is None:
            df = df[~df.index.isin(drop_index)]
        df_all.append(df)

    df_all = pd.concat(df_all)
    df_all.reset_index(level=0, inplace=True)
    df_all.reset_index(level=0, inplace=True)

    df_all.to_csv('/content/drive/My Drive/data_int_{}/'.format(t_interval)+ 'metro_formated_data.csv', index=False)

    return list(df_all['station_id'].unique())

In [5]:
def make_graph_weights(graphs, stations_include, norm='median'):

    # Create index that contains all station pairs that ever appears in the raw data files
    # and apply this index to all graphs.
    mega_index = graphs[0].index
    for graph in graphs[1:]:
        mega_index = mega_index.union(graph.index)
    g_expand = [g.reindex(mega_index, fill_value=0) for g in graphs]
    cat_graph = pd.concat(g_expand).reset_index()

    cat_graph = cat_graph.loc[cat_graph['StartStation Id'].isin(stations_include)]
    cat_graph = cat_graph.loc[cat_graph['EndStation Id'].isin(stations_include)]

    cat_graph_all = cat_graph.groupby(['StartStation Id', 'EndStation Id']).sum()
    cat_graph_all = cat_graph_all.rename(columns={0: 'total'})

    all_df = []
    #Create another feature in the form of percentage of usage.
    for s_ind, df_slice in cat_graph_all.groupby('StartStation Id'):
        mean_slice = df_slice['total'].sum()
        df_slice.loc[:, 'average_percent'] = 100.0 * df_slice['total'] / mean_slice
        all_df.append(df_slice)
    cat_graph_all = pd.concat(all_df)

    return cat_graph_all


In [6]:
def transform_data(raw_data_file_paths, time_interval_size, lower_t_bound, upper_t_bound,
         duration_upper, stations_exclude, station_filename):
    # Create a time interval id mapping to real-world times
    tinterval = time_bins(lower_t_bound, upper_t_bound, time_interval_size)

    tinterval.to_csv('/content/drive/My Drive/data_int_{}/tinterval.csv'.format(time_interval_size))

    # Load station dataset
    stations_const = pd.read_csv(station_filename)
    stations_const_ids = set(stations_const['Station_ID'].to_list())

    # Format multiple bike trip data files
    graphs = []
    for k, file_path in enumerate(raw_data_file_paths):
        print('Processing... {}'.format(file_path))
        spatiotemp, graph_weights = format_data(file_path, tinterval, time_interval_size,
                                                       duration_upper, stations_exclude, stations_const_ids)
        spatiotemp.to_csv('/content/drive/My Drive/data_int_{}/data_reformat_{}.csv'.format(time_interval_size,k))
        graphs.append(graph_weights)

    stations_include = merge_data('/content/drive/My Drive/data_int_{}/data_reformat'.format(time_interval_size), k + 1, time_interval_size)
    print(stations_include)

    # Put together the graph weight data
    df_graph_all = make_graph_weights(graphs, stations_include)
    print(df_graph_all)
    df_graph_all.to_csv('/content/drive/My Drive/data_int_{}/graph_weight.csv'.format(time_interval_size))


### 60 min time interval formed

In [7]:
# File paths to raw data files
files = os.listdir('/content/drive/My Drive/Bikedata')
metro_files = ['/content/drive/My Drive/Bikedata/{}'.format(fp) for fp in files]

# Number of minutes of a time interval 
#For this experiment two interval size is tested 30min and 60 min
Interval = 60

# Start and End of the time period of data (YYYY/MM/DD)
start_period = '2019/01/01'
end_period = '2020/01/01'

# To exclude any outliers
limit = 30000

# To remove any bike station. 
station_drop = []

# Bike station in station dataset
station_df = '/content/drive/My Drive/metro-bike-share-stations-2020-04-01.csv'

# Creates a new directory and stores the dataset.
os.mkdir('/content/drive/My Drive/data_int_{}'.format(Interval))
transform_data(metro_files, Interval, start_period, end_period, limit, station_drop, station_df)


Processing... /content/drive/My Drive/Bikedata/metro-bike-share-trips-2019-q1.csv
Processing... /content/drive/My Drive/Bikedata/metro-bike-share-trips-2019-q3.csv
Processing... /content/drive/My Drive/Bikedata/metro-bike-share-trips-2019-q2.csv
Processing... /content/drive/My Drive/Bikedata/metro-bike-share-trips-2019-q4.csv
Processing file pair 0-1 of 4


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Processing file pair 0-2 of 4
Processing file pair 0-3 of 4
Processing file pair 1-2 of 4
Processing file pair 1-3 of 4
Processing file pair 2-3 of 4
[3000, 3005, 3006, 3007, 3008, 3010, 3011, 3013, 3014, 3016, 3018, 3019, 3020, 3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3040, 3042, 3045, 3046, 3047, 3048, 3049, 3051, 3052, 3054, 3056, 3057, 3058, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3074, 3075, 3076, 3077, 3078, 3081, 3082, 4125, 4126, 4127, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4220, 4227, 4245, 4246, 4248, 4249, 4250, 4254, 4266, 4267, 4273, 4275, 4285, 4286, 4293, 4300, 4304, 4306, 4311, 4312, 4313, 4314, 4315, 4322, 4323, 4324, 4325, 4328, 4329, 4330, 4331, 4332, 4333, 4334, 4335, 4336, 4340, 4341, 4342, 4343, 4344, 4345, 4346, 4347, 4348, 4349, 4350, 4351, 4352, 4353, 4354, 4372, 4378, 4379, 4380, 4381, 4382, 4385, 4373, 4374, 4247, 4301, 4302, 4303, 4337, 4383, 4387, 4389, 4390, 4393, 4394, 4395, 439

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                               total  average_percent
StartStation Id EndStation Id                        
3000            3000             154        30.800000
                3005               1         0.200000
                3006               1         0.200000
                3007               2         0.400000
                3010               3         0.600000
...                              ...              ...
4493            4453               2         6.451613
                4455               3         9.677419
                4473               1         3.225806
                4483               3         9.677419
                4493               2         6.451613

[10232 rows x 2 columns]


In [9]:
transform_data = pd.read_csv('/content/drive/My Drive/data_int_60/metro_formated_data.csv')
transform_data.head()

,time_id,station_id,docks_demand,bikes_demand
0,144,3000,0,0
1,145,3000,0,0
2,146,3000,0,0
3,147,3000,0,0
4,148,3000,0,0


In [10]:
time_int_data = pd.read_csv('/content/drive/My Drive/data_int_30/tinterval.csv')
time_int_data.head()

,Unnamed: 0,interval_id,inner_limit,outer_limit
0,0,0,2019-01-01 00:00:00,2019-01-01 00:30:00
1,1,1,2019-01-01 00:30:00,2019-01-01 01:00:00
2,2,2,2019-01-01 01:00:00,2019-01-01 01:30:00
3,3,3,2019-01-01 01:30:00,2019-01-01 02:00:00
4,4,4,2019-01-01 02:00:00,2019-01-01 02:30:00


In [ ]:
time_int_data.tail()

,Unnamed: 0,interval_id,inner_limit,outer_limit
17515,17515,17515,2019-12-31 21:30:00,2019-12-31 22:00:00
17516,17516,17516,2019-12-31 22:00:00,2019-12-31 22:30:00
17517,17517,17517,2019-12-31 22:30:00,2019-12-31 23:00:00
17518,17518,17518,2019-12-31 23:00:00,2019-12-31 23:30:00
17519,17519,17519,2019-12-31 23:30:00,2020-01-01 00:00:00


In [11]:
graph_wt_data = pd.read_csv('/content/drive/My Drive/data_int_60/graph_weight.csv')
graph_wt_data.head()

,StartStation Id,EndStation Id,total,average_percent
0,3000,3000,154,30.8
1,3000,3005,1,0.2
2,3000,3006,1,0.2
3,3000,3007,2,0.4
4,3000,3010,3,0.6
